# 12 "Making class objects act like values"
Basically making a class as nice as the std::string class

## 12.1 A simple string class
Including the vec from the previous chapter

In [1]:
#include <cstddef>
#include <memory>

In [2]:
.rawInput

Using raw input


In [3]:


using std::max;

template <class T> class Vec {
 public:
  typedef T* iterator;
  typedef const T* const_iterator;
  typedef size_t size_type;
  typedef T value_type;
  typedef T& reference;
  typedef const T& const_reference;

  Vec() { create(); }
  explicit Vec(size_type n, const T& t = T()) { create(n, t); }
  template <class In> Vec(In i, In j) { create(i, j); }

  Vec(const Vec& v) { create(v.begin(), v.end()); }
  Vec& operator=(const Vec&);	// as defined in 11.3.2/196
  ~Vec() { uncreate(); }

  T& operator[](size_type i) { return data[i]; }
  const T& operator[](size_type i) const { return data[i]; }

  void push_back(const T& t) {
    if (avail == limit) grow();
    unchecked_append(t);
  }

  size_type size() const { return avail - data; }  // changed

  iterator begin() { return data; }
  const_iterator begin() const { return data; }

  iterator end() { return avail; }                 // changed
  const_iterator end() const { return avail; }     // changed

  void clear() { uncreate(); }
  bool empty() const { return data == avail; }

  template <class In> void insert(iterator, In, In);
  template <class In> void assign(In, In);

 private:
  iterator data;	// first element in the `Vec'
  iterator avail;	// (one past) the last element in the `Vec'
  iterator limit;	// (one past) the allocated memory

  // facilities for memory allocation
  std::allocator<T> alloc;	// object to handle memory allocation

  // allocate and initialize the underlying array
  void create();
  void create(size_type, const T&);
  template <class In> void create(In, In);

  // destroy the elements in the array and free the memory
  void uncreate();

  // support functions for `push_back'
  void grow();
  void unchecked_append(const T&);
};

// Public functions
template <class T> Vec<T>& Vec<T>::operator=(const Vec& rhs) {
  // check for self-assignment
  if (&rhs != this) {
    // free the array in the left-hand side
    uncreate();

    // copy elements from the right-hand to the left-hand side
    create(rhs.begin(), rhs.end());
  }

  return *this;
}

template <class T> template <class In> void Vec<T>::insert(iterator p, In i, In j) {
  size_type new_size = (avail - data) + (j - i);
  iterator new_data = alloc.allocate(new_size);
  uninitialized_copy(data, p, new_data);
  uninitialized_copy(i, j, new_data + (p - data));
  iterator new_avail = uninitialized_copy(p, avail, new_data + (p - data) + (j - i));

  uncreate();
  
  data = new_data;
  avail = new_avail;
  limit = data + new_size;
}

template <class T> template <class In> void Vec<T>::assign(In i, In j) {
  uncreate();
  create(i, j);
}

// Private functions
template <class T> void Vec<T>::create() {
  data = avail = limit = 0;
}

template <class T> void Vec<T>::create(size_type n, const T& val) {
  data = alloc.allocate(n);
  limit = avail = data + n;
  std::uninitialized_fill(data, limit, val);
}

template <class T> template <class In> void Vec<T>::create(In i, In j) {
  data = alloc.allocate(j - i);
  limit = avail = std::uninitialized_copy(i, j, data);
}

template <class T> void Vec<T>::uncreate() {
  if (data) {
    // destroy (in reverse order) the elements that were constructed
    iterator it = avail;
    while (it != data) alloc.destroy(--it);

    // return all the space that was allocated
    alloc.deallocate(data, limit - data);
  }

  // reset pointers to indicate that the `Vec' is empty again
  data = limit = avail = 0;
}

template <class T> void Vec<T>::grow() {
  // when growing, allocate twice as much space as currently in use
  size_type new_size = max(2 * (limit - data), ptrdiff_t(1));

  // allocate new space and copy existing elements to the new space
  iterator new_data = alloc.allocate(new_size);
  iterator new_avail = std::uninitialized_copy(data, avail, new_data);

  // return the old space
  uncreate();

  // reset pointers to point to the newly allocated space
  data = new_data;
  avail = new_avail;
  limit = data + new_size;
}

// assumes `avail' points at allocated, but uninitialized space
template <class T> void Vec<T>::unchecked_append(const T& val) {
  alloc.construct(avail++, val);
}

In [4]:
.rawInput

Not using raw input


## 12.1 continued, now we define our basic Str class

In [5]:
#include <algorithm>
#include <iostream>
#include <cstring>

In [6]:
class Str {
public:
    typedef Vec<char>::size_type size_type;
    Str() = default;
    Str(size_type n, char c) : data(n, c) { }
    Str(const char* cp) {
        std::copy(cp, cp + std::strlen(cp), std::back_inserter(data));
    }
    
    //I didn't realize you could make a class's method and not the class templated, nice!
    template <class In> Str(In b, In e) {
        std::copy(b, e, std::back_inserter(data));
    }
    private:
        Vec<char> data;
};

"In general a class that needs no destructor doesn't need an explicit copy constructor or assignment operator either.

## 12.2 Automatic Conversions

We want the expressions
```
Str s("hello");
Str t = "hello";
t = "wow";
```

to work, but they have different meanings

* constructors also act as _user-defined conversions_
  * this created a temporary unnamed object from the constructor, then uses the assignment operator. Pretty obvious I guess

2 types of conversions: `other -> this` ,  and `this -> other` (this is going to be in 12.5

## 12.3 Str operations

We want to define these with our `Str s` object
```
cin >> s
cout << s
s[i]
s1 + s2
```

how?! with the keyword `operator`

`operator>>` is the name of the function that overloads the input operator, `operator[]` names the index operation, and so on.

* For a binary operation, the left operand is always bound to the first parameter, and hte right operand is bound to the second
  * so for io operators, we need to add an operator for type cin, cout on the lhs

In [7]:
.rawInput

Using raw input


In [8]:
std::istream& operator>>(std::istream&, const Str&);

class Str2 {
    //lol I hate the friend modifier
    friend std::istream& operator>>(std::istream&, Str&);
public:
    typedef Vec<char>::size_type size_type;
    Str2() = default;
    Str2(size_type n, char c) : data(n, c) { }
    Str2(const char* cp) {
        std::copy(cp, cp + std::strlen(cp), std::back_inserter(data));
    }
    
    //I didn't realize you could make a class's method and not the class templated, nice!
    template <class In> Str2(In b, In e) {
        std::copy(b, e, std::back_inserter(data));
    }
    
    //ask chris why we have 2 separate operators here
    char& operator[](size_type i) { return data[i]; };
    const char& operator[](size_type i) const { return data[i]; };
    

    size_type size() const { return data.size(); }
    private:
        Vec<char> data;
};

    //io operators exist outside the class :-(
std::istream& operator>>(std::istream& is, Str2& s){
    s.data.clear();
    
    char c;
    while (is.get(c) && isspace(c))
        ;
    if (is) {
        do s.data.push_back(c);
        while (is.get(c) && !isspace(c));
        
        if (is)
            is.unget();
    }
    return is;
}
std::ostream& operator<<(std::ostream &os, const Str& s){
    for (Str2::size_type i = 0; i != s.size(); ++i)
        os << s[i];
    return os;
}

input_line_7:31:7: error: 'data' is a private member of 'Str2'
    s.data.clear();
      ^
input_line_7:26:19: note: declared private here
        Vec<char> data;
                  ^
input_line_7:37:14: error: 'data' is a private member of 'Str2'
        do s.data.push_back(c);
             ^
input_line_7:26:19: note: declared private here
        Vec<char> data;
                  ^
input_line_7:46:40: error: no member named 'size' in 'Str'
    for (Str2::size_type i = 0; i != s.size(); ++i)
                                     ~ ^
input_line_7:47:16: error: type 'const Str' does not provide a subscript operator
        os << s[i];
              ~^~


ename: evalue

In [ ]:
.rawInput

* define += first and then +
* define ++ first and then ++(lol)

### mixed type expressions
```
    "Hello" + StrGuy + "\n!";
```
^since these binary operators are left-associative, this is handles like
```
    ("Hello" + StrGuy) + "\n!";
```

and then the compiler converts "Hello" into an Str before doing the + operator

* if you really care a lot about performance you have to create operators for every combination of operands

* If a class supports conversions, then it is usually good practice to define binary operators as nonmember functions. That is, both sides of the operand are potentially results of automatic conversions, whereas binary operators as a member restricts the lhs to being already a class type. When the programmer types
* "compound"/assymetric operators like += should be a member of their class


## 12.4 Some conversions are hazardous

* yeah so after all that, this is why we have the `explicit` keyword, e.g. for constructors that take a size, so we don't get a compiling `StrObj + 4` which doesn't make any sense

## 12.5 Conversion Operators

* you can define explicit conversion operators, which say how to convert an object from its type to a target type. Conversion operators must be defined as members of a class.

```
class Student_info(){
public:
    operator double() const; //conversion **to** a double class
}
```
* this is what's done in all those cases with `cin >> x; if (cin) { //stuff }, == if (cin >> x) { //stuff }`

### Automatic Conversion to char* and ptr types is fraight with memory management pitfalls

